<a href="https://colab.research.google.com/github/NkosinathiNtuli-Signalcompvision/Pytorch-computer-/blob/main/Intro_To_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip -q mydata.zip -d mydata


unzip:  cannot find or open mydata.zip, mydata.zip.zip or mydata.zip.ZIP.


In [ ]:
class DataLoad:
    def __init__(self, datapath=None, outdir=None, classes=None, target_size=(224, 224)):
        self.datapath = datapath
        self.outdir = outdir
        self.CLASSES = classes
        self.target_size = target_size
        logging.info(f'DataLoad instantiated with datapath={self.datapath}, outdir={self.outdir}, target_size={self.target_size}')

    def build(self):
        X, y = [], []
        class_names = sorted(os.listdir(self.datapath)) if self.CLASSES is None else self.CLASSES

        for idx, cls in enumerate(class_names):
            cls_path = Path(self.datapath, cls)
            if not cls_path.is_dir():
                continue
            for file in cls_path.glob("**/*.png"):
                img = plt.imread(file)
                if len(img.shape) == 2:  # grayscale
                    img = np.expand_dims(img, axis=-1)
                img_resized = tf.image.resize(img, self.target_size).numpy()
                X.append(img_resized)
                y.append(idx)

        np.savez(self.outdir, X=np.array(X), y=np.array(y), C=class_names)
        logging.info(f"Processed {len(X)} images into {self.outdir}")

    def load(self):
        if not Path(self.outdir).is_file():
            logging.warning(f"{self.outdir} not found. Building dataset.")
            self.build()
        with np.load(self.outdir, allow_pickle=True) as data:
            self.X = data['X']
            self.y = data['y']
            self.CLASSES = list(data['C'])
        self.X_ = self.standardize()
        logging.info(f"Loaded {self.X.shape[0]} samples from {self.outdir}")

    def standardize(self):
        return np.divide((self.X - self.X.mean(axis=0)), self.X.std(axis=0),
                         out=np.zeros_like((self.X - self.X.mean(axis=0))),
                         where=self.X.std(axis=0)!=0)


class SaliencyHandler:
    """ Handles loading models and computing saliency maps with proper input size. """
    def __init__(self, models_folder, data_folder):
        self.models_folder = models_folder
        self.data_folder = data_folder
        self.MODELS = [DenseNet121, EfficientNetV2S, InceptionV3, InceptionResNetV2, Xception,
                       ResNet50, ResNet50V2, VGG16, VGG19]

    def get_input_shape(self, model_class):
        if model_class in [EfficientNetV2S, DenseNet121, ResNet50, ResNet50V2, VGG16, VGG19]:
            return (224, 224, 3)
        elif model_class in [InceptionV3, InceptionResNetV2, Xception]:
            return (299, 299, 3)
        else:
            raise ValueError(f"Unknown model class: {model_class}")

    def load_data_for_model(self, model_class):
        input_shape = self.get_input_shape(model_class)
        dataset_file = Path(self.data_folder, f"data_{input_shape[0]}x{input_shape[1]}.npz")
        data_loader = DataLoad(datapath=self.data_folder, outdir=dataset_file, target_size=input_shape[:2])
        data_loader.load()
        return data_loader

    def run_saliency_loop(self):
        for model_class in self.MODELS:
            logging.info(f"Processing saliency for {model_class.__name__}")

            # Load dataset adapted for this model
            data_loader = self.load_data_for_model(model_class)

            # Load model from saved folder
            model_path = Path(self.models_folder, f"{model_class.__name__}.keras")
            model = tf.keras.models.load_model(model_path)

            # Here you would call your saliency function using model and data_loader.X_
            # For example: compute_saliency(model, data_loader.X_, data_loader.y)



# Start here


In [ ]:
!rm -rf /content/drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import logging
import time
from pathlib import Path
from typing import Callable, Sequence, Optional

import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import (
    VGG16, VGG19, ResNet50, Xception, ResNet50V2,
    InceptionV3, DenseNet121, EfficientNetV2S, InceptionResNetV2
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from scipy import interpolate, stats
from PIL import Image
import io

# ----------------------------
# Logging setup
# ----------------------------
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# ----------------------------
# PIC Metric & Image Utils
# ----------------------------
def estimate_image_entropy(im_orig: np.ndarray) -> float:
    p = np.histogram(im_orig, bins=256, range=(im_orig.min(), im_orig.max()), density=True)[0]
    return stats.entropy(p, base=2)

def estimate_entropy(image: np.ndarray) -> float:
    if image.dtype in [np.float32, np.float64]:
        image = (image - np.min(image)) / (np.max(image) - np.min(image)) * 255
        image = image.astype(np.uint8)
    if len(image.shape) == 2:
        pil_image = Image.fromarray(image)
    elif len(image.shape) == 3 and image.shape[2] == 1:
        pil_image = Image.fromarray(image[:, :, 0])
    else:
        pil_image = Image.fromarray(image)
    buffer = io.BytesIO()
    pil_image.save(buffer, format='webp', lossless=True, quality=100)
    length = buffer.tell()
    buffer.close()
    return length

def create_blurred_image(full_img: np.ndarray, pixel_mask: np.ndarray, method: str = 'linear') -> np.ndarray:
    img_type = full_img.dtype
    if full_img.ndim == 2:
        full_img = np.expand_dims(full_img, axis=2)
    channels = full_img.shape[2]

    # always include corners
    pixel_mask = pixel_mask.copy()
    h, w = pixel_mask.shape
    pixel_mask[[0, 0, h-1, h-1], [0, w-1, 0, w-1]] = True

    mean_color = np.mean(full_img, axis=(0,1))
    if np.all(pixel_mask):
        return full_img

    blurred_img = full_img * np.expand_dims(pixel_mask, axis=2).astype(np.float32)

    for c in range(channels):
        points = np.argwhere(pixel_mask)
        values = full_img[:, :, c][tuple(points.T)]
        unknown_points = np.argwhere(~pixel_mask)
        interpolated = interpolate.griddata(points, values, unknown_points, method=method, fill_value=mean_color[c])
        blurred_img[:, :, c][tuple(unknown_points.T)] = interpolated

    if full_img.shape[2] == 1:
        blurred_img = blurred_img[:, :, 0]
    if np.issubdtype(img_type, np.integer):
        blurred_img = np.round(blurred_img)
    return blurred_img.astype(img_type)


from collections import namedtuple
PicMetricResult = namedtuple("PicMetricResult", ["curve_x", "curve_y", "blurred_images", "predictions", "thresholds", "auc"])

def compute_pic_metric(
    img: np.ndarray,
    saliency_map: np.ndarray,
    random_mask: np.ndarray,
    pred_func: Callable[[np.ndarray], Sequence[float]],
    saliency_thresholds: Sequence[float],
    keep_monotonous: bool = True,
    num_data_points: int = 1000,
    experiment: Optional[str] = "base",
):
    blurred_images = []
    predictions = []
    entropy_pred_tuples = []

    fully_blurred_img = create_blurred_image(img, random_mask)
    if experiment == "base":
        orig_entropy = estimate_image_entropy(img)
        fully_blurred_entropy = estimate_image_entropy(fully_blurred_img)
    elif experiment == "kapis":
        orig_entropy = estimate_entropy(img)
        fully_blurred_entropy = estimate_entropy(fully_blurred_img)
    else:
        orig_entropy = (estimate_entropy(img) + estimate_image_entropy(img))/2
        fully_blurred_entropy = (estimate_entropy(fully_blurred_img) + estimate_image_entropy(fully_blurred_img))/2

    orig_pred = pred_func(img[np.newaxis, ...])[0]
    fully_blurred_pred = pred_func(fully_blurred_img[np.newaxis, ...])[0]

    blurred_images.append(fully_blurred_img)
    predictions.append(fully_blurred_pred)

    max_pred = 0.0
    for threshold in saliency_thresholds:
        quant = np.quantile(saliency_map, 1 - threshold)
        mask = np.logical_or(saliency_map >= quant, random_mask)
        b_img = create_blurred_image(img, mask)
        if experiment == "base":
            entropy = estimate_image_entropy(b_img)
        elif experiment == "kapis":
            entropy = estimate_entropy(b_img)
        else:
            entropy = (estimate_entropy(b_img) + estimate_image_entropy(b_img))/2
        pred = pred_func(b_img[np.newaxis, ...])[0]

        normalized_entropy = np.clip((entropy - fully_blurred_entropy)/(orig_entropy - fully_blurred_entropy), 0.0, 1.0)
        normalized_pred = np.clip((pred - fully_blurred_pred)/(orig_pred - fully_blurred_pred), 0.0, 1.0)
        max_pred = max(max_pred, normalized_pred)
        if keep_monotonous:
            entropy_pred_tuples.append((normalized_entropy, max_pred))
        else:
            entropy_pred_tuples.append((normalized_entropy, normalized_pred))

        blurred_images.append(b_img)
        predictions.append(pred)

    # interpolate PIC curve
    entropy_pred_tuples.append((0.0, 0.0))
    entropy_pred_tuples.append((1.0, 1.0))
    x_data, y_data = zip(*entropy_pred_tuples)
    interp = interpolate.interp1d(x_data, y_data)
    curve_x = np.linspace(0.0, 1.0, num_data_points, endpoint=False)
    curve_y = np.array([interp(x) for x in curve_x])
    curve_x = np.append(curve_x, 1.0)
    curve_y = np.append(curve_y, 1.0)
    auc = np.trapz(curve_y, curve_x)
    thresholds = [0.0] + list(saliency_thresholds) + [1.0]

    blurred_images.append(img)
    predictions.append(orig_pred)

    return PicMetricResult(curve_x, curve_y, blurred_images, predictions, thresholds, auc)


# ----------------------------
# Data Loader
# ----------------------------
class DataLoad:
    def __init__(self, datapath=None, outdir=None, classes=None, target_size=(224, 224)):
        self.datapath = datapath
        self.outdir = outdir
        self.CLASSES = classes
        self.target_size = target_size
        logging.info(f'DataLoad instantiated with datapath={datapath}, outdir={outdir}, target_size={target_size}')

    def build(self):
        X, y = [], []
        class_names = sorted(os.listdir(self.datapath)) if self.CLASSES is None else self.CLASSES
        for idx, cls in enumerate(class_names):
            cls_path = Path(self.datapath, cls)
            if not cls_path.is_dir():
                continue
            for file in cls_path.glob("**/*.png"):
                img = plt.imread(file)
                if len(img.shape) == 2:
                    img = np.expand_dims(img, axis=-1)
                img_resized = tf.image.resize(img, self.target_size).numpy()
                X.append(img_resized)
                y.append(idx)
        np.savez(self.outdir, X=np.array(X), y=np.array(y), C=class_names)
        logging.info(f"Processed {len(X)} images into {self.outdir}")

    def load(self):
        if not Path(self.outdir).is_file():
            logging.warning(f"{self.outdir} not found. Building dataset.")
            self.build()
        with np.load(self.outdir, allow_pickle=True) as data:
            self.X = data['X']
            self.y = data['y']
            self.CLASSES = list(data['C'])
        self.X_ = self.standardize()
        logging.info(f"Loaded {self.X.shape[0]} samples from {self.outdir}")

    def standardize(self):
        return np.divide((self.X - self.X.mean(axis=0)), self.X.std(axis=0),
                         out=np.zeros_like((self.X - self.X.mean(axis=0))),
                         where=self.X.std(axis=0)!=0)


# ----------------------------
# Saliency Handler
# ----------------------------
class SaliencyHandler:
    def __init__(self, models_folder, data_folder):
        self.models_folder = models_folder
        self.data_folder = data_folder
        self.MODELS = [DenseNet121, EfficientNetV2S, InceptionV3, InceptionResNetV2, Xception,
                       ResNet50, ResNet50V2, VGG16, VGG19]

    def get_input_shape(self, model_class):
        if model_class in [EfficientNetV2S, DenseNet121, ResNet50, ResNet50V2, VGG16, VGG19]:
            return (224, 224, 3)
        elif model_class in [InceptionV3, InceptionResNetV2, Xception]:
            return (299, 299, 3)
        else:
            raise ValueError(f"Unknown model class: {model_class}")

    def load_data_for_model(self, model_class):
        input_shape = self.get_input_shape(model_class)
        dataset_file = Path(self.data_folder, f"data_{input_shape[0]}x{input_shape[1]}.npz")
        data_loader = DataLoad(datapath=self.data_folder, outdir=dataset_file, target_size=input_shape[:2])
        data_loader.load()
        return data_loader

    def run_saliency_loop(self, saliency_func: Callable):
        for model_class in self.MODELS:
            logging.info(f"Processing saliency for {model_class.__name__}")

            data_loader = self.load_data_for_model(model_class)
            model_path = Path(self.models_folder, f"{model_class.__name__}.keras")
            model = tf.keras.models.load_model(model_path)

            # Compute saliency maps for all images
            saliency_maps = [saliency_func(model, img) for img in data_loader.X_]

            # Compute PIC metrics
            results = []
            for img, sal_map, label in zip(data_loader.X_, saliency_maps, data_loader.y):
                random_mask = np.zeros_like(sal_map, dtype=bool)
                pic_result = compute_pic_metric(
                    img=img,
                    saliency_map=sal_map,
                    random_mask=random_mask,
                    pred_func=lambda x: model(x, training=False).numpy()[0],
                    saliency_thresholds=np.linspace(0.1, 0.9, 9)
                )
                results.append(pic_result)

            logging.info(f"Finished saliency and PIC computation for {model_class.__name__}")


# ----------------------------
# Example Usage
# ----------------------------
if __name__ == "__main__":
    models_folder = '/content/drive/MyDrive/Signal Processing ML/Audio Mel-spectrogram/Study sites/Port of St Francies/Models_LR1_0.001_Reg_0.01/'
    data_folder = '/content/drive/MyDrive/Signal Processing ML/Audio Mel-spectrogram/Study sites/Port of St Francies/Test/'

    handler = SaliencyHandler(models_folder=models_folder, data_folder=data_folder)

    # Define a dummy saliency function (replace with your actual implementation)
    def dummy_saliency_func(model, img):
        # just return a random map with same shape as image
        return np.random.rand(*img.shape[:2])

    handler.run_saliency_loop(dummy_saliency_func)


ValueError: File not found: filepath=/content/drive/MyDrive/Signal Processing ML/Audio Mel-spectrogram/Study sites/Port of St Francies/Models_LR1_0.001_Reg_0.01/DenseNet121.keras. Please ensure the file is an accessible `.keras` zip file.

# The previous code is data loader and pIC matric

In [ ]:
import click
import logging
import os
from pathlib import Path
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import cv2
import tensorflow as tf

from data import DataLoad
from utils import Utils
from model import Models
from Download import downloader
from saliencyAnalysis import Saliency, Main
import saliency.core as saliency
from saliency.metrics import pic
from pic import (
    compute_pic_metric, estimate_image_entropy, create_blurred_image,
    estimate_entropy, estimate_image_avg_entropy, compute_pic_metric_flag
)
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore
from tf_keras_vis.utils import normalize
from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.gradcam_plus_plus import GradcamPlusPlus
from tf_keras_vis.scorecam import Scorecam

# Ignore numpy warnings
np.seterr(divide='ignore', invalid='ignore')

CUSTOM_DATASETS = ["./Data/brainTumorDataPublic", "./Data/COVID-19_Radiography_Dataset"]
FONT_SIZE = 14


# ------------------------------
# Utility functions for plotting
# ------------------------------
def show_image(im, title='', ax=None):
    if ax is None:
        fig, ax = plt.subplots(figsize=(12, 6))
    ax.axis('off')
    ax.imshow(im, cmap='gray')
    ax.set_title(title, fontsize=FONT_SIZE)


def show_grayscale_image(im, title='', ax=None):
    if ax is None:
        plt.figure()
    plt.axis('off')
    plt.imshow(im, cmap=plt.cm.gray, vmin=0, vmax=1)
    plt.title(title, fontsize=FONT_SIZE)


def show_curve_xy(x, y, title='PIC', label=None, color='blue', ax=None):
    if ax is None:
        fig, ax = plt.subplots(figsize=(12, 6))
    auc = np.trapz(y) / y.size
    label = f'{label}, AUC={auc:.3f}'
    ax.plot(x, y, label=label, color=color)
    ax.set_title(title, fontsize=FONT_SIZE)
    ax.set_xlim([0.0, 1.1])
    ax.set_ylim([-0.1, 1.1])
    ax.set_xlabel('Normalized estimation of entropy', fontsize=FONT_SIZE)
    ax.set_ylabel('Predicted score', fontsize=FONT_SIZE)
    ax.legend(fontsize=FONT_SIZE)
    ax.grid(True)


def show_curve(compute_pic_metric_result, title='PIC', label=None, color='blue', ax=None):
    show_curve_xy(compute_pic_metric_result.curve_x,
                  compute_pic_metric_result.curve_y,
                  title=title, label=label, color=color, ax=ax)


def show_blurred_images_with_scores(compute_pic_metric_result):
    images_to_display = compute_pic_metric_result.blurred_images
    scores = compute_pic_metric_result.predictions
    thresholds = compute_pic_metric_result.thresholds

    nrows = (len(images_to_display) - 1) // 5 + 1
    fig, ax = plt.subplots(nrows=nrows, ncols=5, figsize=(20, 20 / 5 * nrows))
    for i, im in enumerate(images_to_display):
        row = i // 5
        col = i % 5
        title = f'Score: {scores[i]:.3f}\nThreshold: {thresholds[i]:.3f}'
        show_image(im, title=title, ax=ax[row, col])


# ------------------------------
# Prediction functions
# ------------------------------
def create_predict_function_softmax(class_idx, model):
    def predict(image_batch):
        score = model(image_batch)[:, class_idx]
        return score.numpy()
    return predict


def create_predict_function_accuracy(class_idx, model):
    def predict(image_batch):
        scores = model(image_batch)
        arg_max = np.argmax(scores, axis=1)
        accuracy = arg_max == class_idx
        return np.ones_like(arg_max) * accuracy
    return predict


# ------------------------------
# Main PIC computation function
# ------------------------------
def compute_pic_score(dl, model_indx, masked=True, ds="BrainTumor", experiments="base", n_samples=1000):
    models = Main.load_models(n=3, file_path=f"./Data/{ds}_Evaluation_Results.csv")
    model = tf.keras.models.load_model(f"./Models/{ds}_{models[model_indx]}.keras")
    logging.info("Computing saliency masks...")

    idxs = Saliency.pick_indices(dl, n=3)
    sa = Saliency(model, dl, idxs, masked=masked, dataset=ds)

    guided_ig = saliency.GuidedIG()
    gradient_saliency = saliency.GradientSaliency()
    saliency_thresholds = [0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.07, 0.10, 0.13, 0.21, 0.34, 0.5, 0.75]

    # Initialize result lists
    gig_aic, rnd_aic, gig_sic, rnd_sic = [], [], [], []
    vanilla_aic, smoothgrad_aic, vanilla_sic, smoothgrad_sic = [], [], [], []
    xrai_aic, xrai_sic, gradcam_aic, gradcam_sic = [], [], [], []
    gradcamplusplus_aic, gradcamplusplus_sic, scorecam_aic, scorecam_sic = [], [], [], []

    for idx, im_orig in enumerate(dl.X_):
        replace2linear = ReplaceToLinear()
        score = CategoricalScore([dl.y[idx]])
        predictions, prediction_class, call_model_args, pred_prob = sa.predict(im_orig)

        # Create a random mask for PIC calculation
        random_mask = pic.generate_random_mask(im_orig.shape[0], im_orig.shape[1], fraction=0.01)
        fully_blurred_img = create_blurred_image(full_img=im_orig, pixel_mask=random_mask)

        if experiments == "base":
            original_img_entropy = estimate_image_entropy(im_orig)
            fully_blurred_img_entropy = estimate_image_entropy(fully_blurred_img)
        elif experiments == "kapis":
            original_img_entropy = estimate_entropy(im_orig)
            fully_blurred_img_entropy = estimate_entropy(fully_blurred_img)
        else:
            original_img_entropy = estimate_image_avg_entropy(im_orig)
            fully_blurred_img_entropy = estimate_image_avg_entropy(fully_blurred_img)

        pred_func_sic = create_predict_function_softmax(prediction_class, model)
        pred = pred_func_sic(im_orig[np.newaxis, ...])
        pred_blurred = pred_func_sic(fully_blurred_img[np.newaxis, ...])

        if pred > pred_blurred and original_img_entropy > fully_blurred_img_entropy:
            baseline = np.zeros(im_orig.shape)

            # Compute Guided IG saliency
            guided_ig_mask_3d = guided_ig.GetMask(
                im_orig, sa.call_model_function, call_model_args, x_steps=25,
                x_baseline=baseline, max_dist=1.0, fraction=0.5)
            guided_ig_mask_grayscale = saliency.VisualizeImageGrayscale(guided_ig_mask_3d)

            # Integrated gradients
            integrated_gradients = saliency.IntegratedGradients()
            vanilla_mask = saliency.VisualizeImageGrayscale(
                integrated_gradients.GetMask(im_orig, sa.call_model_function, call_model_args, x_steps=25,
                                             x_baseline=baseline, batch_size=20))
            smoothgrad_mask = saliency.VisualizeImageGrayscale(
                integrated_gradients.GetSmoothedMask(im_orig, sa.call_model_function, call_model_args, x_steps=25,
                                                    x_baseline=baseline, batch_size=20))

            # XRAI
            xrai_object = saliency.XRAI()
            xrai_attributions = xrai_object.GetMask(im_orig, sa.call_model_function, call_model_args, batch_size=20)
            if len(xrai_attributions.shape) == 2:
                xrai_attributions = np.expand_dims(xrai_attributions, axis=2)

            # GradCAM & GradCAM++
            gradcam_map = normalize(Gradcam(model, model_modifier=replace2linear, clone=True)(score, im_orig, penultimate_layer=-1))
            gradcamplusplus_map = normalize(GradcamPlusPlus(model, model_modifier=replace2linear, clone=True)(score, im_orig, penultimate_layer=-1))
            scorecam_map = normalize(Scorecam(model)(score, im_orig, penultimate_layer=-1))

            # Compute PIC metrics for SIC and AIC
            # [Abbreviated here for brevity; you can call your compute_pic_metric blocks as in your original code]

        if len(gig_aic) >= n_samples:
            break

    return gig_aic, rnd_aic, gig_sic, rnd_sic, \
           vanilla_aic, smoothgrad_aic, vanilla_sic, smoothgrad_sic, \
           xrai_aic, xrai_sic, gradcam_aic, gradcam_sic, \
           gradcamplusplus_aic, gradcamplusplus_sic, scorecam_aic, scorecam_sic


# ------------------------------
# CLI entry point
# ------------------------------
@click.command()
@click.option("--ds", default="BrainTumor", help="Dataset to use for training the model.")
@click.option("--experiments", default="base", help="Experiment type (base/kapis/etc).")
@click.option("--n_samples", default=1000, help="Number of samples to run the experiment on.")
def main(ds, experiments, n_samples):
    dl = DataLoad() if ds == "BrainTumor" else DataLoad(datapath=Path(CUSTOM_DATASETS[1]),
                                                         outdir=f'{CUSTOM_DATASETS[1]}.npz', segmented=False)
    dl.load(masked=True if ds == "BrainTumor" else False)
    model_indx = 0

    results = compute_pic_score(dl, model_indx, masked=True, ds=ds, experiments=experiments, n_samples=n_samples)
    print("Computation complete, results ready for plotting and CSV export.")

    # [You can now call your plotting and CSV-saving routines as in the original code.]

if __name__ == "__main__":
    main()


In [ ]:
#=============================
# Saliency Analysis Pipeline with PIC & AIC
#=============================

import logging
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
import saliency.core as saliency
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore
from tf_keras_vis.utils import normalize
from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.gradcam_plus_plus import GradcamPlusPlus
from tf_keras_vis.scorecam import Scorecam

from data import DataLoad   # Your dataset loader
from utils import PackageManager

logging.basicConfig(level=logging.INFO)
PackageManager.install_and_import('saliency')
logging.info("Packages imported successfully")

#=============================
# Saliency Class
#=============================
class Saliency:

    def __init__(self, model, dl, idx, masked=True, dataset="dataset") -> None:
        self.model = model
        self.dl = dl
        self.idx = idx
        self.masked = masked
        self.dataset = dataset
        self.class_idx_str = 'class_idx_str'

    # Model wrapper for saliency computation
    def call_model_function(self, images, call_model_args=None, expected_keys=None):
        target_class_idx = call_model_args[self.class_idx_str]
        images = tf.convert_to_tensor(images)
        with tf.GradientTape() as tape:
            if expected_keys == [saliency.base.INPUT_OUTPUT_GRADIENTS]:
                tape.watch(images)
                output_layer = self.model(images)[:, target_class_idx]
                gradients = np.array(tape.gradient(output_layer, images))
                return {saliency.base.INPUT_OUTPUT_GRADIENTS: gradients}
            else:
                conv_layer, output_layer = self.model(images)
                gradients = np.array(tape.gradient(output_layer, conv_layer))
                return {saliency.base.CONVOLUTION_LAYER_VALUES: conv_layer,
                        saliency.base.CONVOLUTION_OUTPUT_GRADIENTS: gradients}

    @staticmethod
    def PreprocessImage(im):
        return tf.keras.applications.xception.preprocess_input(im)

    @staticmethod
    def pick_indices(dl, n=3):
        sidx = []
        labels = []
        while len(labels) < n:
            idx = np.random.randint(0, dl.X_.shape[0])
            label = dl.y[idx]
            if label not in labels:
                sidx.append(idx)
                labels.append(label)
        return sidx

    def inference(self, idx):
        im = np.expand_dims(self.dl.X_[idx], axis=0)
        predictions = self.model(im)
        prediction_class = np.argmax(predictions[0])
        call_model_args = {self.class_idx_str: prediction_class}
        pred_prob = np.round(predictions[0, prediction_class].numpy(), 2)
        return predictions, prediction_class, call_model_args, pred_prob

    def analyze(self):
        data = {'img': [], 'predictions': [], 'prediction_class': [], 'call_model_args': [],
                'pred_prob': [], 'actual': [], 'idx': []}
        if self.masked:
            data.update({'tumorBorder': [], 'original': []})

        vizresult = {'Image': [], 'VG': [], 'SmoothGrad': [], 'IG': [], 'XRAI_Full': [],
                     'Fast_XRAI': [], 'VIG': [], 'GIG': [], 'Blur_IG': [],
                     'GradCAM': [], 'GradCAM++': [], 'ScoreCAM': []}

        mycollection = []

        for idx in self.idx:
            predictions, prediction_class, call_model_args, pred_prob = self.inference(idx)
            data['img'].append(self.dl.X_[idx])
            data['predictions'].append(predictions)
            data['prediction_class'].append(prediction_class)
            data['call_model_args'].append(call_model_args)
            data['pred_prob'].append(pred_prob)
            data['actual'].append(self.dl.y[idx])
            data['idx'].append(idx)

            if self.masked:
                data['tumorBorder'].append(self.dl.Z[idx])
                data['original'].append(self.dl.A[idx])
                vizresult['Image'].append(self.dl.A[idx])

            im = self.dl.X_[idx]

            # --- Vanilla Gradient & SmoothGrad ---
            gradient_saliency = saliency.GradientSaliency()
            vanilla_mask_3d = gradient_saliency.GetMask(im, self.call_model_function, call_model_args)
            smoothgrad_mask_3d = gradient_saliency.GetSmoothedMask(im, self.call_model_function, call_model_args)
            vizresult['VG'].append(saliency.VisualizeImageGrayscale(vanilla_mask_3d))
            vizresult['SmoothGrad'].append(saliency.VisualizeImageGrayscale(smoothgrad_mask_3d))
            mycollection.extend([saliency.VisualizeImageGrayscale(vanilla_mask_3d),
                                 saliency.VisualizeImageGrayscale(smoothgrad_mask_3d)])

            # --- Integrated Gradients & Guided IG ---
            ig = saliency.IntegratedGradients()
            guided_ig = saliency.GuidedIG()
            baseline = np.zeros(im.shape)
            ig_mask = ig.GetMask(im, self.call_model_function, call_model_args, x_steps=25, x_baseline=baseline)
            guided_ig_mask = guided_ig.GetMask(im, self.call_model_function, call_model_args, x_steps=25, x_baseline=baseline)
            mycollection.extend([saliency.VisualizeImageGrayscale(ig_mask),
                                 saliency.VisualizeImageGrayscale(guided_ig_mask)])

            # --- Blur IG ---
            blur_ig = saliency.BlurIG()
            blur_mask = blur_ig.GetMask(im, self.call_model_function, call_model_args)
            mycollection.append(saliency.VisualizeImageGrayscale(blur_mask))

            # --- XRAI ---
            xrai = saliency.XRAI()
            xrai_mask = xrai.GetMask(im, self.call_model_function, call_model_args)
            mycollection.append(xrai_mask)

            # --- GradCAM, GradCAM++, ScoreCAM ---
            replace2linear = ReplaceToLinear()
            score = CategoricalScore([self.dl.y[idx]])
            gradcam = Gradcam(self.model, model_modifier=replace2linear, clone=True)
            cam = normalize(gradcam(score, im, penultimate_layer=-1))
            gradcampp = GradcamPlusPlus(self.model, model_modifier=replace2linear, clone=True)
            cam_pp = normalize(gradcampp(score, im, penultimate_layer=-1))
            scorecam_obj = Scorecam(self.model)
            cam_sc = normalize(scorecam_obj(score, im, penultimate_layer=-1))
            mycollection.extend([cam, cam_pp, cam_sc])

        return mycollection, data, vizresult

    def plot_results(self, mycollection, data, fmt="tiff", show=False):
        logging.info("Plotting saliency maps...")
        # Use your existing plot_results() logic here
        pass

#=============================
# Main Class: Top-N Models
#=============================
class Main:

    @staticmethod
    def load_models(file_path="./Data/Evaluation_Results.csv", n=3):
        df = pd.read_csv(file_path).nlargest(n, 'f1_score')
        return [df.iloc[i, 1] for i in range(df.shape[0])]

#=============================
# PIC Metrics & AIC
#=============================
def compute_pic_aic(model, dl, idx):
    """
    Computes Pixel Importance Curve (PIC) and Accuracy Information Curve (AIC)
    """
    logging.info("Computing PIC and AIC metrics...")
    pic_scores = []
    aic_scores = []

    for i in idx:
        im = np.expand_dims(dl.X_[i], axis=0)
        pred = model(im)
        pred_class = np.argmax(pred[0])
        # Simple PIC: fraction of top-k salient pixels
        # Example: use GradientSaliency
        gs = saliency.GradientSaliency()
        mask = gs.GetMask(dl.X_[i], lambda x, **kw: {saliency.base.INPUT_OUTPUT_GRADIENTS: np.array(x)},
                          { 'class_idx_str': pred_class })
        pic = np.sum(mask) / np.prod(mask.shape)
        pic_scores.append(pic)

        # Simple AIC: sum of correct predictions weighted by PIC
        correct = int(pred_class == dl.y[i])
        aic_scores.append(correct * pic)

    avg_pic = np.mean(pic_scores)
    avg_aic = np.mean(aic_scores)
    logging.info(f"Avg PIC: {avg_pic:.4f}, Avg AIC: {avg_aic:.4f}")
    return avg_pic, avg_aic

#=============================
# Execution Script
#=============================
if __name__ == "__main__":
    dl = DataLoad()
    dl.load()  # Load your dataset
    idx = Saliency.pick_indices(dl, n=3)

    models = Main.load_models(n=2)  # Top 2 models
    for m in tqdm(models):
        logging.info(f"Loading model: {m}")
        model = tf.keras.models.load_model(f"./Models/{m}")
        sa = Saliency(model, dl, idx)
        mycollection, data, vizresult = sa.analyze()
        sa.plot_results(mycollection, data)

        # Compute PIC & AIC for research metrics
        pic, aic = compute_pic_aic(model, dl, idx)
        logging.info(f"Saliency analysis completed for {model.name} | PIC={pic:.4f}, AIC={aic:.4f}")


In [ ]:
!git clone https://github.com/PAIR-code/saliency.git
!cd saliency && pip install -e .


Cloning into 'saliency'...
remote: Enumerating objects: 4468, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 4468 (delta 4), reused 3 (delta 3), pack-reused 4452 (from 3)
Receiving objects: 100% (4468/4468), 323.89 MiB | 15.16 MiB/s, done.
Resolving deltas: 100% (249/249), done.
Obtaining file:///content/saliency
  Preparing metadata (setup.py) ... done
  Running setup.py develop for saliency


In [ ]:
import os
import random
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
import logging
import pip
import importlib
import sys
import subprocess
import logging
import cv2

class Utils:
    """
        This class impliments utility functions to allow experiment to be carried out successfully. These are custom functions particular to this codebase
    """
    def __init__(self) -> None:
        pass


    @staticmethod
    def resize(x: np.ndarray) -> np.ndarray:
        # Assuming x is a grayscale image
        resized_img = cv2.resize(x, (225, 225), interpolation=cv2.INTER_LINEAR)
        expanded_img = np.expand_dims(resized_img, axis=-1)
        return expanded_img

    @staticmethod
    def plot_samples(X: np.ndarray,y: np.ndarray, Z: np.ndarray, CLASSES: list,idx,figsize=(8,8),save=False, show=True, segmented=False, masked=True, filename="Sample", fmt="tiff"):
        """
            plot_samples plots randomly selected samples from the dataset
            :X ndarray of the selected mri slices
            : y ndarray of class labels 0 meningioma, 1 glioma, 2 pituitary tumor
            : CLASSES is a list of the above mentioned class strings
            :idx ndarray of n sample indices
        """
        if(segmented):
            fig,axs = plt.subplots(4,4, figsize=figsize)
            fig.subplots_adjust(hspace=.5, wspace=.001)
            for i, ax in zip(list(range(0,len(idx))),axs.ravel()):
                for j in range(Z[idx[i]].shape[0]-1):
                    ax.imshow(X[idx[i]])
                    ax.scatter(Z[idx[i]][j],  Z[idx[i]][j+1],marker=".", color="red", alpha=0.6)
                    ax.set_title(CLASSES[y[idx[i]]].capitalize())
                    ax.set_axis_off()
            if (show):
                plt.show()
            if(save):
                fig.savefig(f'./Figures/{filename}.{fmt}',bbox_inches ="tight",dpi=300)
            plt.close(fig)
        else:
            fig,axs = plt.subplots(4,4, figsize=figsize)
            fig.subplots_adjust(hspace=.5, wspace=.001)
            for i, ax in zip(list(range(0,len(idx))),axs.ravel()):
                ax.imshow(np.squeeze(X[idx[i]]), cmap='gray',interpolation='none') # I would add interpolation='none'
                if(masked):
                 ax.imshow(Z[idx[i]], cmap='jet', alpha=0.4,interpolation='none') # interpolation='none'
                ax.set_title(CLASSES[y[idx[i]]].capitalize().replace("_"," "))
                ax.set_axis_off()
            if (show):
                plt.show()
            if(save):
                fig.savefig(f'./Figures/{filename}.{fmt}',bbox_inches ="tight",dpi=300)
            plt.close(fig)
    @staticmethod
    def project2D(X_,y,CLASSES,figname='XMRI_TSNE',save=False, show=True,fmt="tiff"):
        """
            The functions takes a standardized dataset X_ and projects it to 2D using TSNE technique for visualization.
        """
        X_embedded = TSNE(n_components=2, learning_rate='auto',init='random').fit_transform(X_.reshape(X_.shape[0],-1).astype('float64'))
        logging.info("TNSE embedding created successfully. Displaying the 2D projection onto a scatterplot.")
        # #plotting results
        fig = plt.figure(1,figsize=(8,4))
        scatter = plt.scatter(X_embedded[:,0],X_embedded[:,1], c=list(y))
        plt.xlabel(r'$x_1$')
        plt.ylabel(r'$x_2$')
        plt.legend(handles=scatter.legend_elements()[0], labels=CLASSES,bbox_to_anchor=(1.0, 1.0))
        if(save):
            plt.savefig(f'./Figures/{figname}.{fmt}', bbox_inches ="tight", dpi=300)
            logging.info(f"Figure ./Figures/{figname}.{fmt} written successfully.")
        if(show):
            plt.show()
        plt.close(fig)

    @staticmethod
    def plot_confusion_matrix(y_true: np.ndarray, y_pred: np.ndarray,CLASSES: list, save=True, filename='confusion_matrix',figsize=(8,6), fmt="tiff") -> None:
        """
            : y_true the ground truth class labels
            : y_pred the predicted class labels
            : CLASSES, the list of classes

            The function plots a confusion matrix of the model predictions.
        """
        fig = plt.figure(1,figsize=figsize)
        CLASSES = [str(i).capitalize() for i in CLASSES]
        df_cm = pd.DataFrame(confusion_matrix(y_true, y_pred), index = [i for i in CLASSES],columns = [i for i in CLASSES])
        sns.heatmap(df_cm, annot=True, cmap = 'Blues',fmt='g')
        if(save):
            plt.savefig(f'./Figures/{filename}.{fmt}', bbox_inches ="tight", dpi=300)
        plt.close(fig)
        logging.info(f"Plot saved successfully to ./Figures/{filename}.{fmt}")

    @staticmethod
    def plot_evaluation(dft, save=True, filename="Train_Time_Accuracy", fmt="tiff"):
        #@title Plotting Models test performance
        # sort df by Count column
        dft['model'] = dft['model'].apply(lambda x: str(x).replace("_"," ").title())
        pd_df = dft.sort_values(['f1_score']).reset_index(drop=True)
        plt.figure(figsize=(8,6))
        # plot barh chart with index as x values
        ax = sns.barplot(pd_df.model, pd_df.f1_score)
        ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(np.round(float(x),1))))
        ax.set_xlabel("Model Architecture",fontsize=12)
        ax.set_ylabel(r"$F_1$ Score",fontsize=12)
        # add proper Dim values as x labels
        ax.set_xticklabels(pd_df.model)
        for item in ax.get_xticklabels(): item.set_rotation(90)
        for i, v in enumerate(pd_df["f1_score"].iteritems()):
            ax.text(i ,v[1], "{:,}".format(np.round(v[1],2)), color='b', va ='bottom', rotation=40, fontsize=12)
        plt.tight_layout()
        plt.savefig(f'./Figures/{filename}.{fmt}', bbox_inches ="tight", dpi=300)
        plt.show()
        plt.close()

    @staticmethod
    def create_embedding(model_name: str, X_data: np.ndarray):
        logging.info(f"Loading {model_name.capitalize()} model from ./Models/{model_name}.keras")
        model = tf.keras.models.load_model(f"./Models/{model_name}.keras",compile=True)
        repmodel  = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output, name = model.name)
        logging.info("Model loaded and re-initialized successfully")
        logging.info(f"Starting inference on {X_data.shape[0]} samples")
        X_hat =  repmodel.predict(X_data)
        logging.info("Inference completed successfully")
        return X_hat

class PackageManager:
    def __init__(self) -> None:
        pass
    @staticmethod
    def install_and_import(package: str):
        try:
            importlib.import_module(package)
        except ImportError:
            pip.main(['install', package])
        finally:
            globals()[package] = importlib.import_module(package)

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)


In [ ]:
# ========================
# Install required packages
# ========================
!pip install git+https://github.com/PAIR-code/saliency.git
!pip install tf-keras-vis

# ========================
# Standard imports
# ========================
import PIL.Image
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
from matplotlib import pyplot as plt
from tqdm import tqdm
import logging

# Saliency imports
import saliency  # core saliency package
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore
from tf_keras_vis.utils import normalize
from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.gradcam_plus_plus import GradcamPlusPlus
from tf_keras_vis.scorecam import Scorecam

logging.basicConfig(level=logging.INFO)
logging.info("Packages imported successfully")

# ========================
# Saliency class
# ========================
class SaliencyAnalysis:
    def __init__(self, model, dl, idx, masked=True, dataset_name="dataset"):
        self.model = model
        self.dl = dl
        self.idx = idx
        self.masked = masked
        self.dataset_name = dataset_name
        self.class_idx_str = 'class_idx_str'

    # Compute prediction gradients
    def call_model_function(self, images, call_model_args=None, expected_keys=None):
        target_class_idx = call_model_args[self.class_idx_str]
        images = tf.convert_to_tensor(images)
        with tf.GradientTape() as tape:
            if expected_keys == [saliency.base.INPUT_OUTPUT_GRADIENTS]:
                tape.watch(images)
                output_layer = self.model(images)
                output_layer = output_layer[:, target_class_idx]
                gradients = np.array(tape.gradient(output_layer, images))
                return {saliency.base.INPUT_OUTPUT_GRADIENTS: gradients}
            else:
                conv_layer, output_layer = self.model(images)
                gradients = np.array(tape.gradient(output_layer, conv_layer))
                return {
                    saliency.base.CONVOLUTION_LAYER_VALUES: conv_layer,
                    saliency.base.CONVOLUTION_OUTPUT_GRADIENTS: gradients
                }

    @staticmethod
    def preprocess_image(im):
        return tf.keras.applications.xception.preprocess_input(im)

    @staticmethod
    def pick_indices(dl, n=3):
        sidx = []
        labels = []
        while len(labels) < n:
            idx = np.random.randint(0, dl.X_.shape[0])
            label = dl.y[idx]
            if label not in labels:
                sidx.append(idx)
                labels.append(label)
        return sidx

    def predict(self, im):
        im = np.expand_dims(im, axis=0)
        predictions = self.model(im)
        prediction_class = np.argmax(predictions[0])
        call_model_args = {self.class_idx_str: prediction_class}
        pred_prob = np.round(predictions[0, prediction_class].numpy(), 2)
        return predictions, prediction_class, call_model_args, pred_prob

    def inference(self, idx):
        im = np.expand_dims(self.dl.X_[idx], axis=0)
        predictions = self.model(im)
        prediction_class = np.argmax(predictions[0])
        call_model_args = {self.class_idx_str: prediction_class}
        pred_prob = np.round(predictions[0, prediction_class].numpy(), 2)
        return predictions, prediction_class, call_model_args, pred_prob

    def analyze(self):
        """Compute various saliency maps"""
        mycollection = []
        data = {'img': [], 'predictions': [], 'prediction_class': [], 'call_model_args': [], 'pred_prob': [], 'actual': [], 'idx': []}

        for idx in self.idx:
            predictions, prediction_class, call_model_args, pred_prob = self.inference(idx)
            data['img'].append(self.dl.X_[idx])
            data['predictions'].append(predictions)
            data['prediction_class'].append(prediction_class)
            data['call_model_args'].append(call_model_args)
            data['pred_prob'].append(pred_prob)
            data['actual'].append(self.dl.y[idx])
            data['idx'].append(idx)

            im = self.dl.X_[idx]

            # Vanilla gradient saliency
            gradient_saliency = saliency.GradientSaliency()
            vanilla_mask_3d = gradient_saliency.GetMask(im, self.call_model_function, call_model_args)
            smoothgrad_mask_3d = gradient_saliency.GetSmoothedMask(im, self.call_model_function, call_model_args)
            vanilla_mask = saliency.VisualizeImageGrayscale(vanilla_mask_3d)
            smoothgrad_mask = saliency.VisualizeImageGrayscale(smoothgrad_mask_3d)
            mycollection.extend([vanilla_mask, smoothgrad_mask])

            # Integrated gradients
            integrated_gradients = saliency.IntegratedGradients()
            baseline = np.zeros(im.shape)
            ig_mask = integrated_gradients.GetMask(im, self.call_model_function, call_model_args, x_steps=25, x_baseline=baseline, batch_size=20)
            ig_mask_grayscale = saliency.VisualizeImageGrayscale(ig_mask)
            mycollection.append(ig_mask_grayscale)

            # Grad-CAM
            replace2linear = ReplaceToLinear()
            score = CategoricalScore([self.dl.y[idx]])
            gradcam = Gradcam(self.model, model_modifier=replace2linear, clone=True)
            cam = normalize(gradcam(score, im, penultimate_layer=-1))
            mycollection.append(cam)

            # Grad-CAM++
            gradcampp = GradcamPlusPlus(self.model, model_modifier=replace2linear, clone=True)
            campp = normalize(gradcampp(score, im, penultimate_layer=-1))
            mycollection.append(campp)

            # Score-CAM
            scorecam_obj = Scorecam(self.model)
            scorecam_map = normalize(scorecam_obj(score, im, penultimate_layer=-1))
            mycollection.append(scorecam_map)

        return mycollection, data

    def plot_results(self, mycollection, data, fmt="png", show=False):
        """Plot computed saliency maps"""
        num_maps = len(mycollection)
        fig, axs = plt.subplots(1, num_maps, figsize=(4*num_maps, 4))
        if num_maps == 1:
            axs = [axs]
        for i in range(num_maps):
            axs[i].imshow(np.squeeze(mycollection[i]), cmap='jet')
            axs[i].axis('off')
        plt.tight_layout()
        if show:
            plt.show()
        plt.savefig(f'{self.dataset_name}_Saliency_Maps.{fmt}', dpi=300)
        plt.close(fig)

# ========================
# Example usage
# ========================
if __name__ == "__main__":
    # Example: replace this with your dataset loader
    # from data import DataLoad
    # dl = DataLoad()
    # dl.load()

    # Dummy example: random data for testing
    class DummyData:
        def __init__(self):
            self.X_ = np.random.rand(3, 224, 224, 3)
            self.y = np.array([0, 1, 2])
            self.CLASSES = ["Class0", "Class1", "Class2"]
    dl = DummyData()

    # Dummy model for testing
    model = tf.keras.applications.Xception(weights=None, input_shape=(224,224,3), classes=3)

    idx = SaliencyAnalysis.pick_indices(dl, n=3)
    sa = SaliencyAnalysis(model, dl, idx)
    mycollection, data = sa.analyze()
    sa.plot_results(mycollection, data, show=True)


  Cloning https://github.com/PAIR-code/saliency.git to /tmp/pip-req-build-mq3ywj8t
  Running command git clone --filter=blob:none --quiet https://github.com/PAIR-code/saliency.git /tmp/pip-req-build-mq3ywj8t
  Resolved https://github.com/PAIR-code/saliency.git to commit 7b72b673fde6680180befd277c1890d4a9de4b48
  Preparing metadata (setup.py) ... done


AttributeError: module 'saliency' has no attribute 'GradientSaliency'

# Salincy appliation

In [ ]:
# ==========================================
# Section 0: Imports & Setup
# ==========================================
import os
import logging
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt

# Saliency imports
import saliency.core as saliency
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore
from tf_keras_vis.utils import normalize
from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.gradcam_plus_plus import GradcamPlusPlus
from tf_keras_vis.scorecam import Scorecam

# Custom modules
from data import DataLoad
from utils import PackageManager

logging.basicConfig(level=logging.INFO)
PackageManager.install_and_import('saliency')

logging.info("All packages imported successfully.")

# ==========================================
# Section 1: Load Data
# ==========================================
def load_dataset(data_path):
    """
    Load your dataset using DataLoad class
    """
    dl = DataLoad(data_path=data_path)
    dl.load()
    logging.info(f"Dataset loaded. Number of samples: {dl.X_.shape[0]}")
    return dl

# ==========================================
# Section 2: Load Models
# ==========================================
def load_top_models(csv_path, models_path, n=2):
    """
    Load top-n models from evaluation CSV
    """
    df = pd.read_csv(csv_path).nlargest(n, 'f1_score')
    models = []
    for i in range(df.shape[0]):
        model_name = df.iloc[i, 1]
        logging.info(f"Loading model: {model_name}")
        model = tf.keras.models.load_model(os.path.join(models_path, model_name))
        models.append(model)
    return models

# ==========================================
# Section 3: Saliency Analysis Class
# ==========================================
class SaliencyAnalysis:
    def __init__(self, model, dl, idx, masked=True, dataset_name="dataset"):
        self.model = model
        self.dl = dl
        self.idx = idx
        self.masked = masked
        self.dataset = dataset_name
        self.class_idx_str = 'class_idx_str'

    # --- Model call for saliency ---
    def call_model_function(self, images, call_model_args=None, expected_keys=None):
        target_class_idx = call_model_args[self.class_idx_str]
        images = tf.convert_to_tensor(images)
        with tf.GradientTape() as tape:
            if expected_keys == [saliency.base.INPUT_OUTPUT_GRADIENTS]:
                tape.watch(images)
                output_layer = self.model(images)
                output_layer = output_layer[:, target_class_idx]
                gradients = np.array(tape.gradient(output_layer, images))
                return {saliency.base.INPUT_OUTPUT_GRADIENTS: gradients}
            else:
                conv_layer, output_layer = self.model(images)
                gradients = np.array(tape.gradient(output_layer, conv_layer))
                return {saliency.base.CONVOLUTION_LAYER_VALUES: conv_layer,
                        saliency.base.CONVOLUTION_OUTPUT_GRADIENTS: gradients}

    # --- Pick random indices ---
    @staticmethod
    def pick_indices(dl, n=3):
        sidx, labels = [], []
        while len(labels) < n:
            idx = np.random.randint(0, dl.X_.shape[0])
            label = dl.y[idx]
            if label not in labels:
                sidx.append(idx)
                labels.append(label)
        return sidx

    # --- Inference ---
    def inference(self, idx):
        im = np.expand_dims(self.dl.X_[idx], axis=0)
        predictions = self.model(im)
        prediction_class = np.argmax(predictions[0])
        call_model_args = {self.class_idx_str: prediction_class}
        pred_prob = np.round(predictions[0, prediction_class].numpy(), 2)
        return predictions, prediction_class, call_model_args, pred_prob

    # --- Saliency computation ---
    def analyze(self):
        logging.info("Starting saliency analysis...")
        mycollection, data, vizresult = [], {'img': [], 'predictions': [], 'prediction_class': [], 'call_model_args': [], 'pred_prob': [], 'actual': [], 'idx': []}, {}
        for idx in self.idx:
            predictions, pred_class, call_model_args, pred_prob = self.inference(idx)
            im = self.dl.X_[idx]

            data['img'].append(im)
            data['predictions'].append(predictions)
            data['prediction_class'].append(pred_class)
            data['call_model_args'].append(call_model_args)
            data['pred_prob'].append(pred_prob)
            data['actual'].append(self.dl.y[idx])
            data['idx'].append(idx)

            # Vanilla Gradient
            gradient_saliency = saliency.GradientSaliency()
            vanilla_mask_3d = gradient_saliency.GetMask(im, self.call_model_function, call_model_args)
            vanilla_mask_grayscale = saliency.VisualizeImageGrayscale(vanilla_mask_3d)
            mycollection.append(vanilla_mask_grayscale)

            # Integrated Gradients
            integrated_gradients = saliency.IntegratedGradients()
            baseline = np.zeros(im.shape)
            vanilla_ig_mask = integrated_gradients.GetMask(im, self.call_model_function, call_model_args,
                                                           x_steps=25, x_baseline=baseline, batch_size=20)
            vanilla_ig_grayscale = saliency.VisualizeImageGrayscale(vanilla_ig_mask)
            mycollection.append(vanilla_ig_grayscale)

            # Grad-CAM
            replace2linear = ReplaceToLinear()
            score = CategoricalScore([self.dl.y[idx]])
            gradcam = Gradcam(self.model, model_modifier=replace2linear, clone=True)
            cam = normalize(gradcam(score, im, penultimate_layer=-1))
            mycollection.append(cam)

        logging.info("Saliency analysis completed.")
        return mycollection, data

    # --- Plot saliency ---
    def plot_saliency(self, mycollection, data, save_path="./Figures"):
        os.makedirs(save_path, exist_ok=True)
        plt.figure(figsize=(12, 4))
        for i, img in enumerate(mycollection):
            plt.subplot(1, len(mycollection), i+1)
            plt.imshow(img, cmap='jet')
            plt.axis('off')
        plt.savefig(os.path.join(save_path, f'{self.dataset}_Saliency.png'), dpi=300)
        plt.show()

# ==========================================
# Section 4: PIC and AIC (Optional)
# ==========================================
# Placeholder for your PIC / AIC computations
def compute_pic_aic(predictions, labels):
    """
    Compute Performance Information Curve (PIC) and Accuracy Information Curve (AIC)
    Example placeholder, replace with actual implementation
    """
    logging.info("Computing PIC and AIC metrics...")
    # Your PIC / AIC computation goes here
    pic_curve = np.random.rand(10)  # Dummy
    aic_curve = np.random.rand(10)  # Dummy
    return pic_curve, aic_curve

def plot_pic_aic(pic_curve, aic_curve):
    plt.figure(figsize=(8,4))
    plt.plot(pic_curve, label="PIC")
    plt.plot(aic_curve, label="AIC")
    plt.title("PIC & AIC Curves")
    plt.xlabel("Threshold")
    plt.ylabel("Metric")
    plt.legend()
    plt.show()

# ==========================================
# Section 5: Main Execution
# ==========================================
if __name__ == "__main__":
    # Paths to your data and models
    DATA_PATH =  '/content/drive/MyDrive/Signal Processing ML/Audio Mel-spectrogram/Study sites/Port of St Francies/Test/'
    MODELS_PATH = '/content/drive/MyDrive/Signal Processing ML/Audio Mel-spectrogram/Study sites/Port of St Francies/Models_LR1_0.001_Reg_0.01/'


    CSV_PATH =  '/content/drive/MyDrive/Signal Processing ML/Audio Mel-spectrogram/Study sites/Port of St Francies/'

    # Load data
    dl = load_dataset(DATA_PATH)

    # Pick sample indices for saliency
    idx = SaliencyAnalysis.pick_indices(dl, n=3)

    # Load top models
    models = load_top_models(CSV_PATH, MODELS_PATH, n=2)

    for model in models:
        # Saliency analysis
        sa = SaliencyAnalysis(model, dl, idx)
        mycollection, data = sa.analyze()
        sa.plot_saliency(mycollection, data)

        # PIC & AIC
        pic_curve, aic_curve = compute_pic_aic(data['predictions'], data['actual'])
        plot_pic_aic(pic_curve, aic_curve)


ModuleNotFoundError: No module named 'data'

In [1]:
import torch

In [2]:
!pip list


Package                                  Version
---------------------------------------- --------------------
absl-py                                  1.4.0
absolufy-imports                         0.3.1
accelerate                               1.10.1
aiofiles                                 24.1.0
aiohappyeyeballs                         2.6.1
aiohttp                                  3.13.0
aiosignal                                1.4.0
alabaster                                1.0.0
albucore                                 0.0.24
albumentations                           2.0.8
ale-py                                   0.11.2
alembic                                  1.17.0
altair                                   5.5.0
annotated-types                          0.7.0
antlr4-python3-runtime                   4.9.3
anyio                                    4.11.0
anywidget                                0.9.18
argon2-cffi                              25.1.0
argon2-cffi-bindings              